In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.cross_validation import KFold, train_test_split
from BeautifulSoup import BeautifulSoup
from pymongo import MongoClient
from lxml import etree

In [47]:
#Path of data files
pospath = r'C:\Anaconda\Galvanize\Minimester2-Project\txt_sentoken\pos'
negpath = r'C:\Anaconda\Galvanize\Minimester2-Project\txt_sentoken\neg'

#Other initializations including MongoDB and lists of data
client = MongoClient()
db = client.reviews
collection = db.movies
review = []
opinion = []

#Drop MongoDB data so duplicates aren't loaded
collection.drop()

#loop through neg files
for name in os.listdir(negpath):
    #context manager so files aren't floating around
    with open(negpath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        #remove apostrophes so that contractions are treated as one word
        #otherwise wouldn't, can't, won't etc will all tokenize to t
        review.append(data.replace("'",""))
        opinion.append(False)

#loop through pos files
for name in os.listdir(pospath):
    #context manager so files aren't floating around
    with open(pospath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        review.append(data.replace("'",""))
        opinion.append(True)

df = pd.DataFrame(zip(review,opinion))
df.columns = ['Review','Opinion']
df.head()
result = collection.insert(df.T.to_dict().values())
print collection.count()


2000


In [46]:
#Hyperparameters to tune
#n_samples = 2000
n_features = 100
n_topics = 4
n_top_words = 20

moviesdf = pd.DataFrame(list(collection.find()))
movies = moviesdf.as_matrix()
moviestext = movies[:,1]
     
moviestext_train, moviestext_test = train_test_split(moviestext)
kf = KFold(moviestext_train.shape[0], n_folds=5)
vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

for train_index, test_index in kf:    
    text_tfidf = vectorizer.fit_transform(list(\
            moviestext_train[train_index]))
    print text_tfidf.shape
    nmf = NMF(n_components=n_topics).fit(text_tfidf)
#     print nmf.reconstruction_err_

feature_names = vectorizer.get_feature_names()

for idx, sentiment in enumerate(nmf.components_):
    print "Sentiment #{}".format(idx)
    print(" ".join([feature_names[i]
                    for i in sentiment.argsort()[:-n_top_words - 1:-1]]))
    print

(1200, 100)
(1200, 100)
(1200, 100)
(1200, 100)
(1200, 100)
Sentiment #0
film characters films character really just scenes scene audience best plot good time way performance actually great like make people

Sentiment #1
movie movies good plot seen did like really people funny bad just original characters know thing character little audience actually

Sentiment #2
life story love man family world new young like home does john doesn time old director year role best comedy

Sentiment #3
action like hes bad theres doesnt just dont films good guy big effects movies time plot special better make script



In [44]:
n_topics = 4

def reconst_mse(target, left, right):
    return (np.array(target - left.dot(right))**2).mean()

def describe_nmf_results(document_term_mat, W, H, n_top_words = 15):
    print("Reconstruction error: %f") %(reconst_mse(document_term_mat, W, H))
    for topic_num, topic in enumerate(H):
        print("Topic %d:" % topic_num)
        print(" ".join([feature_words[i] \
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return

moviesdf = pd.DataFrame(list(collection.find()))
movies = moviesdf.as_matrix()
moviestext = movies[:,1]
     
moviestext_train, moviestext_test = train_test_split(moviestext)
kf = KFold(moviestext_train.shape[0], n_folds=5)
vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

for train_index, test_index in kf:    
    nmf = NMF(n_components=n_topics).fit(text_tfidf)
    text_tfidf = vectorizer.fit_transform(list(moviestext_train[train_index]))
    feature_words = vectorizer.get_feature_names()
    
    print("\n\n---------\nsklearn decomposition")
    nmf = NMF(n_components=n_topics)
    W_sklearn = nmf.fit_transform(text_tfidf)
    H_sklearn = nmf.components_
    describe_nmf_results(text_tfidf, W_sklearn, H_sklearn)



---------
sklearn decomposition
Reconstruction error: 0.006186
Topic 0:
film characters character films just really scenes scene audience performance good best great plot like
Topic 1:
movie movies seen good just plot funny characters like people did really know think bad
Topic 2:
action hes theres like bad doesnt dont just big effects good special films guy time
Topic 3:
life love story man new family world young does time doesn like john little role


---------
sklearn decomposition
Reconstruction error: 0.006247
Topic 0:
film characters films character scenes plot really audience just scene story like performance good seen
Topic 1:
movie movies funny good like just bad seen characters plot think comedy know really original
Topic 2:
life love man story world new family john like time young does little doesn people
Topic 3:
action theres doesnt hes bad big dont just effects like special guy good films better


---------
sklearn decomposition
Reconstruction error: 0.006207
Topic 0:
m